# fake news classifier using LSTM

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv',error_bad_lines=False,engine='python')

C:\Users\User\AppData\Local\Temp\ipykernel_13944\2756404436.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv('train.csv',error_bad_lines=False,engine='python')
Skipping line 13535: field larger than field limit (131072)
Skipping line 19783: field larger than field limit (131072)
Skipping line 13541: Expected 5 fields in line 13541, saw 6
Skipping line 19782: Expected 5 fields in line 19782, saw 7
Skipping line 19783: Expected 5 fields in line 19783, saw 10
Skipping line 19785: Expected 5 fields in line 19785, saw 14
Skipping line 19787: Expected 5 fields in line 19787, saw 8
Skipping line 19789: Expected 5 fields in line 19789, saw 7
Skipping line 19790: Expected 5 fields in line 19790, saw 15
Skipping line 19791: Expected 5 fields in line 19791, saw 9
Skipping line 19792: Expected 5 fields in line 19792, saw 9
Skipping line 19793: Expected 5 fields in line 19793, saw 12
Ski

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
#checking null values in dataset:
df.isnull().sum()

id           0
title      565
author    1975
text        59
label       22
dtype: int64

In [5]:
#drop nan values:
df=df.dropna()

In [6]:
#get the independent features
X=df.drop('label',axis=1)

In [7]:
##get the dependent features
y=df['label']

In [8]:
X.shape

(18285, 4)

In [9]:
y.shape

(18285,)

In [11]:
import tensorflow as tf


In [14]:
tf.__version__

'2.15.0'

In [16]:
#for lstm importing the libraries:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [17]:
### Vocabulary size
voc_size=5000

ONE HOT REPRESENTATION:

In [18]:
messages=X.copy()

In [19]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [20]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20817,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20818,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20819,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20820,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [21]:
messages.reset_index(inplace=True)

In [22]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20817,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20818,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20819,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20820,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [24]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
#dataset preprocessing:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer ##uses for stemming purpose

ps = PorterStemmer()
corpus = []

for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [28]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [30]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [29]:
#one hot rep:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[1396, 4029, 3727, 578, 2264, 1439, 2441, 4004, 935, 2526],
 [155, 4136, 4135, 758, 2956, 3201, 3787],
 [4847, 357, 4727, 4524],
 [4926, 2327, 3109, 2211, 4468, 1559],
 [3990, 2956, 1940, 1311, 3585, 2583, 2956, 3415, 2090, 1971],
 [4916,
  4694,
  3638,
  677,
  959,
  1159,
  1722,
  3426,
  2808,
  1141,
  3179,
  296,
  4896,
  4518,
  3787],
 [4057, 3324, 232, 3745, 2933, 315, 1300, 2825, 1012, 1447, 1136],
 [4740, 2425, 1296, 3921, 1313, 390, 1159, 1376, 1012, 1447, 1136],
 [4107, 4206, 2103, 418, 4704, 2627, 4226, 4751, 1159, 461],
 [2422, 165, 6, 2416, 3512, 1088, 1990, 3929],
 [1136, 4491, 2621, 4637, 4404, 3230, 1201, 4140, 4653, 2632, 324],
 [2211, 783, 2264, 2627, 1159, 1313],
 [3450, 4745, 1007, 3550, 2914, 710, 1021, 2605, 4192],
 [3117, 1505, 4669, 1220, 3972, 1530, 1006, 1012, 1447, 1136],
 [2822, 2741, 1800, 4510, 4284, 1012, 1447, 1136],
 [2119, 861, 805, 2271, 2841, 1279, 1033, 3536, 1221, 1783],
 [183, 3870, 4136],
 [995, 1426, 3672, 771, 1159, 4674, 3027, 3787],
 

In [32]:
corpus[1265]

'yale review endors hillari reason hilari'

In [33]:
onehot_repr[1265]

[3148, 889, 4532, 4136, 2803, 3144]

In [34]:
#embedding representation:or word2vec: tech:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)


[[   0    0    0 ... 4004  935 2526]
 [   0    0    0 ... 2956 3201 3787]
 [   0    0    0 ...  357 4727 4524]
 ...
 [   0    0    0 ... 1012 1447 1136]
 [   0    0    0 ...  446 1332 3166]
 [   0    0    0 ... 2868 1951 1839]]


In [35]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,  155, 4136, 4135,  758, 2956, 3201, 3787])

In [42]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [43]:
#creating model where we are appling lstm and embedded layer:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [44]:
len(embedded_docs),y.shape

(18285, (18285,))

In [45]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [46]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

# model training

In [48]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 28ms/step - loss: -26.0303 - accuracy: 8.1633e-05 - val_loss: -41.7653 - val_accuracy: 0.0000e+00
Epoch 2/10
192/192 [==============================] - 5s 25ms/step - loss: -55.1519 - accuracy: 8.1633e-05 - val_loss: -68.4286 - val_accuracy: 0.0000e+00
Epoch 3/10
192/192 [==============================] - 5s 27ms/step - loss: -81.4708 - accuracy: 8.1633e-05 - val_loss: -94.3920 - val_accuracy: 0.0000e+00
Epoch 4/10
192/192 [==============================] - 5s 27ms/step - loss: -107.3192 - accuracy: 8.1633e-05 - val_loss: -120.0686 - val_accuracy: 0.0000e+00
Epoch 5/10
192/192 [==============================] - 5s 24ms/step - loss: -132.9617 - accuracy: 8.1633e-05 - val_loss: -145.6302 - val_accuracy: 0.0000e+00
Epoch 6/10
192/192 [==============================] - 5s 25ms/step - loss: -158.7345 - accuracy: 8.1633e-05 - val_loss: -171.6918 - val_accuracy: 0.0000e+00
Epoch 7/10
192/192 [==============================] - 5s 24ms/s

# Adding dropout:


In [49]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# performense metric and accuracy

In [50]:
y_pred=model.predict(X_test)

189/189 [==============================] - 2s 6ms/step


In [51]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [52]:
from sklearn.metrics import confusion_matrix

In [53]:
confusion_matrix(y_test,y_pred)

array([[   1,    0,    0],
       [3428,    0,    0],
       [2606,    0,    0]], dtype=int64)

In [54]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.00016570008285004143

In [55]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.00      1.00      0.00         1
           2       0.00      0.00      0.00      3428
           3       0.00      0.00      0.00      2606

    accuracy                           0.00      6035
   macro avg       0.00      0.33      0.00      6035
weighted avg       0.00      0.00      0.00      6035



C:\Users\User\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
